In [ ]:
# Télécharger les ressources necéssaires pour utiliser le script
import pip
#!pip install python-binance
#!pip install pytz

import os
import pytz
import pandas as pd
from datetime import date, timedelta, datetime
from market_imbalance import MarketImbalance, MarketCandle, create_market_candle, find_imbalances_after_fall, find_imbalances_after_rise
#from analysis_functions import find_imbalances_after_rise, find_imbalance_after_fall
from interact_with_binance import fetch_ohlcv, adjust_timestamps_to_local
from binance.client import Client



In [ ]:
# Récupérer les clés API à partir des variables d'environnement
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

# Initialiser le client Binance
client = Client(api_key, api_secret)

server_time = pd.to_datetime(client.get_server_time()['serverTime'], unit='ms')
my_time = datetime.now(pytz.timezone('Europe/Paris'))
print(server_time)
print(my_time)

# Récupérer l'historique des prix pour une paire donnée depuis son listing sur la plateforme
df = fetch_ohlcv(client, symbol = "BTCUSDT", interval=Client.KLINE_INTERVAL_1MINUTE, from_date= "24 Mar, 2024") #"1 Jan, 2015"
df = adjust_timestamps_to_local(df, 'timestamp')

In [ ]:
imbalances_after_fall = find_imbalances_after_fall(df)
for imbalance in imbalances_after_fall[-1:]:
    print(imbalance.timestamp, ",", imbalance.open_price, ",", imbalance.close_price, ",", imbalance.delta_to_be_filled_in)
    

In [ ]:
for imbalance in imbalances_after_fall :
        imbalance_start = (imbalance.timestamp)
        #print('imbalance timestamp :', imbalance.timestamp)
        #print('imbalance open price :', imbalance.open_price)
        #print('imbalance close price :', imbalance.close_price)
        
        candles_after_imbalance = df[df['timestamp'] > imbalance_start] # filter df on candles after the imbalance
        highest_price_from_now = imbalance.open_price
        
        
        for index, row in candles_after_imbalance.iterrows() :
            #print('row index :', index)
            #print('highest price from now :', highest_price_from_now)
            if row['high'] >= imbalance.close_price :
                MarketImbalance.mark_as_full_filled(imbalance, row['timestamp']) # update is_full_filled=False to True,and replace was_fullfilled_at=None by was_fullfilled_at=candle['timestamp']
                MarketImbalance.update_time_to_be_fullfilled(imbalance, row['timestamp'])

            elif row['high'] > highest_price_from_now : 
                highest_price_from_now = row['high']
                
                
# Maintenant j'ai ma fonction qui vérifie si les imbalances sont comblés.
# Regardons si leur statut est cahngé dans la classe.         

fullfilled_imbalances = list(filter(lambda imbalance: imbalance.is_full_filled, imbalances_after_fall))
for imb in fullfilled_imbalances[-3:] :
    print('imbalance that started on', imb.timestamp, 'was full filled on', imb.was_fullfilled_at, 'within', imb.was_fullfilled_at - imb.timestamp, 'open price =', imb.open_price, 'close_price =', imb.close_price)

In [ ]:
# il faut récupérer le dataframe avec l'ensemble des bougies du marché.
# ensuite il faut récupérer les imbalances
# pour chaque imbalance, nous allons vérifier les éléments suivants

def get_fullfilled_candles(df_of_market_data, imbalances) :
    for imbalance in imbalances[-1:] :
        imbalance_start = (imbalance.timestamp)
        print('imbalance open price :', imbalance.open_price)
        print('imbalance close price :', imbalance.close_price)
        print('imbalance timestamp :', imbalance.timestamp)
        
        candles_after_imbalance = df_of_market_data[df_of_market_data['timestamp'] > imbalance_start] #filter df on candles after the imbalance
        
        highest_price_from_now = imbalance.open_price
        last_row_index = df_of_market_data.iloc[-1]
        print(last_row_index)
        
        for index, row in candles_after_imbalance.iterrows() :
            while imbalance.is_full_filled == False :
                print('row index :', index)
                print('highest price from now :', highest_price_from_now)
                if row['high'] >= imbalance.close_price :
                    MarketImbalance.mark_as_full_filled(imbalance, row['timestamp']) # update is_full_filled=False to True,and replace was_fullfilled_at=None by was_fullfilled_at=candle['timestamp']
    
                elif row['high'] > highest_price_from_now : 
                    highest_price_from_now = row['high']
            
            
        
    print(imbalance.is_full_filled)
    #return candles_after_imbalance
        
get_fullfilled_candles(df_of_market_data=df, imbalances=imbalances_after_fall)


# Imbalance.candles_of_fullfilling

In [ ]:
imbalances_after_rise = find_imbalances_after_rise(df)
for imbalance in imbalances_after_rise:
    print(imbalance.timestamp, ",", imbalance.open_price, ",", imbalance.close_price, ",", imbalance.delta_to_be_filled_in)

In [ ]:
Imbalance_candles = pd.DataFrame([previous_candles, current_candles, next_candles], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
Imbalance_candles

In [ ]:
# Identifier tous les imbalances
all_imbalances = find_imbalances_after_rise(df) + find_imbalances_after_fall(df)
print(len(all_imbalances))

# Return all imbalances
all_imbalances

In [ ]:
# retourne une liste des jours où il y a eu des imbalances
list_of_imbalance_start = []
for imbalance in all_imbalances :
    list_of_imbalance_start.append(imbalance[0])


In [ ]:
# retourne une liste des jours où il y a eu des imbalances qui n'ont pas été comblés par la suite
random_imbalance_after_a_rise = find_imbalances_after_rise(df)

df_exp = pd.DataFrame(random_imbalance_after_a_rise, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
df_exp['timestamp'] = pd.to_datetime(df_exp['timestamp'], unit='ms')
df_exp.set_index('timestamp', inplace=True)

In [ ]:
# un imbalance après une hausse est comblé si :
imbalance_timestamp = timestamp_imb
imbalance_open = n+1[low]
imbalance_close = n-1[high]
imbalance_depth = imbalance_open - imbalance_close
is_imbalance_filled = "function is_imbalance_filled_(imbalance, toutes les occurences dérrière)"

    """ pour chaque bougie après l'imbalance, regarder le prix le plus bas de la bougie. 
        si le prix le plus est égal ou inférieur à imbalance_close,
        alors is_imbalance_full_filled == true
        & imbalance_was_fullfilled_at = bougie[timestamp]
        
        si le prix le plus bas est strictement inférieur à imbalance_open
        is_imbalance_partially_filled == true
        
        
        mettre à jour la caractéristique de l'imbalance 
    """


In [ ]:
# code à modifier afin de pouvoir travailler avec tradingview sur des timeframes en heure et en minutes.
# Je suis limité à 5000 barres dans tradingview. Je dois donc réduire la liste des imbalances à ceux réalisés dans les 5000 derniers jours à partir d'aujourd'hui
# Get today's date
today = date.today()

# Calculate the date 5000 days ago
days_ago = today - timedelta(days=5000)
print(days_ago)

# Filter dates within the last 5000 days
recent_dates = [d for d in list_of_imbalance_start if d >= days_ago]

# Print recent dates
print("Recent dates within the last 5000 days:")
for d in recent_dates:
    print(d)